Hello there

In [1]:
from classes.Grid import Grid
from classes.util import animate
import matplotlib.pyplot as plt

In [2]:
dim = (9,9)
data = {"start" : (0, 0), "terminal" : [], "wall" : []}
for i in [0, 1, 2, 3, 5, 6, 7, 8]:
    data["terminal"].append(((4, i), -10))
data["terminal"].append(((8, 8), 10))
easy = Grid(dim, data, living_reward=0)

### Disclaimer
This notebook gives a short overview over certain topics, but ultimately I'm more interested with concepts than proving every statement.

## Markov Decision Process, Intro (MDP)

In Markov Decision Processes, we say there is an underlying model with a state space $S$ that has all the possible states, actions $A$ that tell us what actions can be taken at some state, a transition function $T(s, a) = P(s\prime | s, a)$, that tells us the probability distribution of the next state $s\prime$ after taking some action at some state, and a reward function, $R(s, a, s\prime)$, that returns the reward for our agent after a transition. Altogether, MDP's are defined with a set {$S, A, T(s, a), R(s, a, s\prime)$}.

In Gridworld, we have a simple Markov Decision Process that can be represented as a 2D board. This is useful for examining some basic RL algorithms, because the finite state space and action space are easy to understand. We can then add complexity later. For example, we will assume in our first few examples that our MDP is deterministic: that is, $T(s, a) = s\prime$, and there is no randomness in what happens after taking an action. We will also assume our reward function is deterministic (*something that is less important, as randomly given rewards can usually be substituted by their expectation in most contexts*).

When we are trying to solve an MDP, we often desire an optimal policy $\pi^*$, that tells us what the best action to take is at some state $s$. The policy is essentially a function $\pi(s) \rightarrow a$. Then, our agent— whether some robot or sprite, or perhaps even a human playing our Gridworld— can simply follow the policy to get the best result. 

Now we have described our assumptions, let's dive into the algorithms.

## 1. Value Iteration

The intuition behind Value Iteration is that for any state $s \in S$ for our MDP, there is some implied optimal value $V_s^*$, the value that we could gain starting from $s$ and following the optimal policy $\pi^*$. If we could learn all the values $V^*$ for all the states, the optimal policy is simple to extract: Take the action that will lead to the best value. 

Since solving for $V^*$ directly is intractable (whether computationally intensive or infeasible), we perform a recursive update that is given by the Bellman equation (try Wikipedia if you're curious). The Value Iteration update equation is given below.

$$ V_s = \max_a \sum_{s\prime} \mathbb{P}[s\prime | s, a] * (R(s, a, s\prime) + \gamma V_{s\prime})$$

This may be confusing, so to explain the equation plainly: The value of a state, $V_s$, is given by the best action $a$ which returns the best future reward. This reward is given by the weighted sum over the possible results after taking action $a$ from state $s$, the next states $s\prime$. The reward for the action is R(s, a, s\prime) plus the value of the next state, $V_{s\prime}$, and we weight it by the probability of that next state occuring. $\gamma \in [0, 1]$ is a discount factor that essentially tells our agent to prioritize closer rewards compared to faroff rewards; eg. a reward 10 spaces away will be scaled by $\gamma^{10}$.

We run the update over all the states, and repeat until convergence. Convergence to the optimal values $V^*$ is guarenteed. An example is given below; the animation shows the evolution of the values after each update. Note that this naive algorithm has runtime $O(S^2A)$—— we iterate over the state space, we iterate over actions, and each action could possible iterate over all other states. If the degree of your MDP is limited, ie. the max degree in your MDP graph is $d$, then you would have runtime $O(SAd)$.

In [3]:
from classes.ValueAgent import ValueAgent
samples = 20
snell = ValueAgent(easy, epsilon=0, gamma=1)
value_data = [snell.value_matrix]
history = []
for x in range(samples):
    snell.update_all()
    value_data.append(snell.value_matrix.copy())
animate(value_data) # Animation doesn't appear in vscode. Run jupyter notebook to see animation.

In [6]:
snell = ValueAgent(easy, epsilon=0, gamma=0.9)
runtime = snell.run_value_iteration()
print(runtime)

18


In [7]:
from classes.PolicyAgent import PolicyAgent
mex = PolicyAgent(easy, epsilon=0.5, gamma=0.9)
runtime = mex.run_policy_iteration()
print(runtime)

16
